# Reading into histograms

In [1]:
import os, sys
import json
import numpy as np
import ROOT

print('Modules loaded.')

Welcome to JupyROOT 6.26/10
Modules loaded.


In [2]:
jsonfile2018 = '../../InputJsons/lumidata_2018.json'

with open(jsonfile2018,'r') as infile: filedict = json.load(infile)
    
jobnames = ["hist_2LSS_SE2_Oct03_mm"]
histname = 'nnscore_qcd_vlld_combined'

signals = ['VLLD_ele', 'VLLD_mu']

In [3]:
sigdict = {
    'VLLD_ele': {
        'M100': {'mass': 100, 'xsec': 16.9, 'ngen': 110871},
        'M200': {'mass': 200, 'xsec': 1.36, 'ngen': 73730},
        'M300': {'mass': 300, 'xsec': 0.291, 'ngen': 24753},
        'M400': {'mass': 400, 'xsec': 0.0907, 'ngen': 24491},
        'M600': {'mass': 600, 'xsec': 0.0149, 'ngen': 24611},
        'M800': {'mass': 800, 'xsec': 0.00347, 'ngen': 23680},
        'M1000': {'mass': 1000, 'xsec': 0.000971, 'ngen': 24286}
    },
    'VLLD_mu': {
        'M100': {'mass': 100, 'xsec': 16.9, 'ngen': 111926},
        'M200': {'mass': 200, 'xsec': 1.36, 'ngen': 73908},
        'M300': {'mass': 300, 'xsec': 0.291, 'ngen': 25022},
        'M400': {'mass': 400, 'xsec': 0.0907, 'ngen': 24299},
        'M600': {'mass': 600, 'xsec': 0.0149, 'ngen': 24890},
        'M800': {'mass': 800, 'xsec': 0.00347, 'ngen': 24763}
    }
}

bkgdict = {}

def set_last_bin_as_overflow(hst):
    lastBin = hst.GetNbinsX()
    content = hst.GetBinContent(lastBin)
    error = hst.GetBinError(lastBin)
    overflow = hst.GetBinContent(lastBin + 1)
            
    updated_content = content + overflow
    updated_error = (error**2 + overflow**2)**0.5
            
    hst.SetBinContent(lastBin, updated_content)
    hst.SetBinError(lastBin, updated_error)
            
    # Handle underflow:
    content_first = hst.GetBinContent(1)
    error_first = hst.GetBinError(1)
    underflow = hst.GetBinContent(0)
            
    updated_content_first = content_first + underflow
    updated_error_first = (error_first**2 + underflow**2)**0.5
            
    hst.SetBinContent(1, updated_content_first)
    hst.SetBinError(1, updated_error_first)

print('Functions loaded.')

Functions loaded.


In [4]:
for sample, subs in filedict.items():
    if sample not in bkgdict: bkgdict[sample] = {}

    for subsample, lumi in subs.items():
        #if sample not in 'VLLD_mu': continue
        if 'SingleMuon' in sample or 'EGamma' in sample: continue
        #print(sample, subsample, lumi)
        if subsample not in bkgdict[sample]: bkgdict[sample][subsample] = {}
        
        yields = []
        errors = []
        integrals = []

        #Fill-up filedict and signal-dict with more information:
        # Step1: Open the histogram and find out yield and error in in each bins.
        for job in jobnames:
            
            input_dir = os.path.join('../input_hists', job)
            filename = f'hst_{sample}_{subsample}.root'
            filepath = os.path.join(input_dir, filename)
            if not os.path.exists(filepath): continue

            tfile = ROOT.TFile(filepath)
            hist = tfile.Get(histname)

            set_last_bin_as_overflow(hist)
            hist.Scale(59800/lumi)

            hist.Rebin(5)
            integral = hist.Integral()
            integrals.append(integral)

            nbins = hist.GetNbinsX()
            #print(nbins)
            for bin in range(1, nbins + 1):
                yield_value = hist.GetBinContent(bin)
                error_value = hist.GetBinError(bin)
                yields.append(yield_value)
                errors.append(error_value)

            tfile.Close()

        if 'VLL' not in sample:
            bkgdict[sample][subsample]['yields'] = yields
            bkgdict[sample][subsample]['errors'] = errors
            bkgdict[sample][subsample]['integrals'] = integrals

        if 'VLL' in sample:
            if sample not in sigdict:            sigdict[sample] = {}
            if subsample not in sigdict[sample]: sigdict[sample][subsample] = {}
            sigdict[sample][subsample]['yields'] = yields
            sigdict[sample][subsample]['errors'] = errors
            sigdict[sample][subsample]['integrals'] = integrals
            #print(f'Updated dictionary for {sample} {subsample}')
            if sample == 'VLLD_mu' and subsample=='M100': print(yields, errors, integrals)
        

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.604369163513184, 7.5355706214904785, 8.121240615844727, 16.020751953125, 35.8721923828125, 63.90382766723633, 157.65457153320312, 106.15678405761719, 148.54534912109375, 140.87515258789062, 212.55780029296875, 195.47897338867188, 191.01226806640625, 187.77586364746094, 186.9493408203125, 251.87644958496094, 216.72866821289062, 256.46636962890625, 321.1999816894531, 240.8889923095703, 264.70977783203125, 215.229736328125, 293.0583801269531, 262.9051513671875, 230.96966552734375, 162.87310791015625, 210.67849731445312, 216.32095336914062, 105.29951477050781] [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.604369479786646, 7.535570769262841, 8.121240793996508, 11.38460142281825, 16.136382983522207, 22.604669104336637, 35.34644847313183, 28.47979154129845, 34.274036528441194, 33.32769972956097, 40.363258911018434, 38.59216281830593, 38.422313352821575, 38.471462658822674, 37.654736607709715, 43.41636140919447, 40.508673055

### Extracting background yields

In [5]:
combined_bkg_yield = None
combined_bkg_error = None

# Loop through the samples and subsamples in bkgdict
for sample, subs in bkgdict.items():
    if 'VLL' in sample: continue
        
    for subsample, val in subs.items():
        
        # Check if 'yields' and 'errors' keys exist in the current subsample
        if 'yields' not in val or 'errors' not in val:
            print(f"Warning: 'yields' or 'errors' not found for sample {sample}, subsample {subsample}.")
            continue
            
        # Get the yields and errors for the current subsample
        yields = np.array(val['yields'])
        errors = np.array(val['errors'])

        if yields.size == 0 or errors.size == 0: continue

        # Initialize the combined arrays if not already initialized
        if combined_bkg_yield is None:
            combined_bkg_yield = np.zeros_like(yields)
            combined_bkg_error = np.zeros_like(errors)

        # Add yields normally
        combined_bkg_yield += yields

        # Add errors in quadrature
        combined_bkg_error = np.sqrt(combined_bkg_error**2 + errors**2)
        print(f"Total yields and errors calculated for background: {sample} {subsample}")

# Now, `combined_bkg_yield` contains the total yields binwise
# And `combined_bkg_error` contains the total errors binwise
print('\n'+'-'*100)
print("Combined Background Yield:", combined_bkg_yield)
print("Combined Background Error:", combined_bkg_error)

Total yields and errors calculated for background: DYJetsToLL M10to50
Total yields and errors calculated for background: DYJetsToLL M50
Total yields and errors calculated for background: HTbinnedWJets 70to100
Total yields and errors calculated for background: HTbinnedWJets 100to200
Total yields and errors calculated for background: HTbinnedWJets 200to400
Total yields and errors calculated for background: HTbinnedWJets 400to600
Total yields and errors calculated for background: HTbinnedWJets 600to800
Total yields and errors calculated for background: HTbinnedWJets 800to1200
Total yields and errors calculated for background: HTbinnedWJets 1200to2500
Total yields and errors calculated for background: HTbinnedWJets 2500toInf
Total yields and errors calculated for background: QCD_MuEnriched 20to30
Total yields and errors calculated for background: QCD_MuEnriched 30to50
Total yields and errors calculated for background: QCD_MuEnriched 50to80
Total yields and errors calculated for background:

### Extracting data yields (Setting it to background for now)

In [6]:
combined_data_yield = combined_bkg_yield
combined_data_error = combined_bkg_error

## Preparing datacard

In [7]:
def prepare_datacard(sigdict, combined_bkg_yield, combined_bkg_error):
    nbins = len(combined_bkg_yield)  # Set nbins based on the size of combined_bkg_yield
    combined_data_yield = combined_bkg_yield
    combined_data_error = combined_bkg_error

    for sample, subs in sigdict.items():
        for subsample, val in subs.items():
            # Prepare a unique output filename
            if sample != 'VLLD_mu': continue
            if subsample != 'M100': continue

            os.makedirs('datacards', exist_ok=True)
            #os.makedirs('yields', exist_ok=True)
            outname = f"datacards/datacard_{sample}_{subsample}.txt"
            #outname_yield = f"yields/datacard_{sample}_{subsample}.txt"

            # Prepare signal data
            sigregs = []
            yields = np.array(val['yields'])
            errors = np.array(val['errors'])

            # If yields are zero, set a small value
            yields[yields == 0] = 1e-7
            
            for i in range(nbins):
                sig = yields[i] if i < len(yields) else 0
                dsig = errors[i] if i < len(errors) else 0

                # Ensure the background is not zero
                bkg = combined_bkg_yield[i] if i < len(combined_bkg_yield) else 0
                dbkg = combined_bkg_error[i] if i < len(combined_bkg_error) else 0
                
                if bkg == 0: continue  # Reject bins with zero background
                
                # Signal-to-background ratio
                stob = sig / np.sqrt(bkg)

                # Create the entry for the datacard
                sigregs.append([i + 1, sig, combined_data_yield[i], bkg, dbkg, stob])

            # Sort by signal-to-background ratio
            sigregs.sort(key=lambda x: x[5], reverse=True)

            # Write the datacard to file
            with open(outname, 'w') as fout:
                fout.write(f"imax {len(sigregs)} \t\t\t # number of channels\n")
                fout.write("jmax 1 \t\t\t # number of backgrounds\n")
                fout.write(f"kmax {len(sigregs)} \t\t\t # number of nuisance parameters\n")
                fout.write("------------\n")

                # Block Data
                fout.write(f"{'bin':<10}")
                for i in range(len(sigregs)):
                    fout.write(f"\t{'bin' + str(i + 1)}")
                fout.write("\n")

                fout.write(f"{'observation':<10}")
                for reg in sigregs:
                    fout.write(f"\t{reg[2]:.2f}")
                fout.write("\n------------\n")

                # Block Signal and Bkg
                fout.write(f"{'bin':<10}")
                for i in range(len(sigregs)):
                    fout.write(f"\t{'bin' + str(i + 1)}\t{'bin' + str(i + 1)}")
                fout.write("\n")

                fout.write(f"{'process':<10}")
                for i in range(len(sigregs)):
                    fout.write(f"\tsig\tbkg")
                fout.write("\n")

                fout.write(f"{'process':<10}")
                for i in range(len(sigregs)):
                    fout.write(f"\t{-1 * (i + 1)}\t{i + 1}")
                fout.write("\n")

                fout.write(f"{'rate':<10}")
                for reg in sigregs:
                    fout.write(f"\t{reg[1]:.2f}\t{reg[3]:.2f}")
                fout.write("\n------------\n")

                # Block dBkg
                for i in range(len(sigregs)):
                    fout.write(f"xs{i + 1}\tlnN")
                    for j in range(len(sigregs)):
                        if j == i:
                            fout.write(f"\t-\t{1 + reg[4] / reg[3]:.2f}")
                        else:
                            fout.write("\t-\t-")
                    fout.write("\n")

In [8]:
# Example usage
prepare_datacard(sigdict, combined_bkg_yield, combined_bkg_error)